In [1]:
%pip install dbfread

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from dbfread import DBF
import psycopg2
import csv
from io import StringIO
from datetime import datetime
from dbfread import DBF

In [3]:
import psycopg2
import psycopg2.extras

# Connect to the PostgreSQL database
conn = psycopg2.connect(
    dbname="bytsscom_unmsm",
    user="postgres",
    password="postgres",
    host="localhost",
    port="5432"
)
conn.autocommit = True
cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)

In [13]:
from decimal import Decimal

In [52]:
def get_consolidado():
    cur.execute(
    """
  SELECT c.ano_eje as  anio,
       '0000'as cod_siaf_area,
       f.siaf_codigo ,
       c.certificado,
       sum(cm.monto_nacional)             AS MONTO_CLASIFICADOR

FROM bytsscom_bytsiaf.certificado c
         INNER JOIN bytsscom_bytsiaf.certificado_fase cf
                    ON c.certificado = cf.certificado
         INNER JOIN bytsscom_bytcore.fuente f
                    ON cf.fuente_financ = f.siaf_codigo
                        AND cf.ano_eje = '2025'
         INNER JOIN bytsscom_bytsiaf.certificado_secuencia cs
                    ON c.certificado = cs.certificado
                        AND cf.secuencia = cs.secuencia
                        AND cs.estado_registro = 'A'
                        AND cs.ano_eje = '2025'
         INNER JOIN bytsscom_bytsiaf.certificado_meta cm
                    ON cf.certificado = cm.certificado
                        AND cf.secuencia = cm.secuencia
                        AND cm.ano_eje = '2025'
         LEFT JOIN bytsscom_bytcore.meta_institucional_siaf mt
                    ON cm.sec_func = mt.sec_func
                    AND mt.id_anio = '2025'::int
         INNER JOIN bytsscom_bytcore.meta_institucional mp
                    ON mt.id_meta_institucional = mp.id_meta_institucional
         INNER JOIN bytsscom_bytcore.clasificador cla
                    ON substring(cm.sys_cod_clasif, 1, 2) = cla.cod_clasif
         LEFT JOIN (SELECT distinct c.id_certificacion,
                                    m.id_area
                    FROM bytsscom_bytsig.vw_certificacion c
                             inner join bytsscom_bytsig.memo_requerimiento m
                                        on c.id_memo_requerimiento = m.id_memo_requerimiento
                    where c.id_anio = 2025
                      AND c.esta_cert = 'A') cert
                   ON cf.sys_id_certificacion = cert.id_certificacion
WHERE c.ano_eje = '2025'
--AND cf.certificado = '0000006928'
GROUP BY c.ano_eje,
         f.siaf_codigo ,
         c.certificado
ORDER BY c.certificado;
    """,)
    return cur.fetchall()
  

In [53]:
resultados = get_consolidado()
print(resultados)
for fila in resultados:
    print(fila)

[['2025', '0000', '00', '0000000001', Decimal('8406955.48')], ['2025', '0000', '00', '0000000002', Decimal('263809773.38')], ['2025', '0000', '00', '0000000003', Decimal('11412972.00')], ['2025', '0000', '00', '0000000004', Decimal('7976388.27')], ['2025', '0000', '00', '0000000005', Decimal('3561790.72')], ['2025', '0000', '00', '0000000006', Decimal('403200.00')], ['2025', '0000', '00', '0000000007', Decimal('51016520.09')], ['2025', '0000', '09', '0000000008', Decimal('78440.00')], ['2025', '0000', '09', '0000000009', Decimal('100000.00')], ['2025', '0000', '09', '0000000010', Decimal('4000.00')], ['2025', '0000', '09', '0000000011', Decimal('20000.00')], ['2025', '0000', '09', '0000000012', Decimal('50000.00')], ['2025', '0000', '09', '0000000013', Decimal('50000.00')], ['2025', '0000', '09', '0000000014', Decimal('10000.00')], ['2025', '0000', '09', '0000000015', Decimal('22000.00')], ['2025', '0000', '09', '0000000016', Decimal('20000.00')], ['2025', '0000', '09', '0000000017', D

In [54]:
import pandas as pd
from psycopg2 import errors  #
pd.set_option('display.float_format', '{:,.2f}'.format)
def get_consolidado_df():
    raw = get_consolidado()
    df = pd.DataFrame(
        raw,
        columns=['anio', 'cod_siaf_area', 'siaf_fuente','certificado','monto_clasificador']
    )
    df['monto_clasificador'] = df['monto_clasificador'].astype(float)
    return df

# Uso
df_1 = get_consolidado_df()
print(df_1)

      anio cod_siaf_area siaf_fuente certificado  monto_clasificador
0     2025          0000          00  0000000001        8,406,955.48
1     2025          0000          00  0000000002      263,809,773.38
2     2025          0000          00  0000000003       11,412,972.00
3     2025          0000          00  0000000004        7,976,388.27
4     2025          0000          00  0000000005        3,561,790.72
...    ...           ...         ...         ...                 ...
2354  2025          0000          09  0000002533           11,296.30
2355  2025          0000          09  0000002536           38,400.00
2356  2025          0000          09  0000002542           90,000.00
2357  2025          0000          00  0000002544           20,000.00
2358  2025          0000          09  0000002545            2,000.00

[2359 rows x 5 columns]


In [67]:
import pandas as pd

def create_insert_consolidado(row):

    def val(v):
        # Si quisieras tratar None o NaN:
        if pd.isna(v):
            return 'NULL'
        return v

    query = f"""
    INSERT INTO sistema_informacion_gerencial.hechos_institucional_consolidados
      (anio, cod_siaf_area, fuente_siaf, num_certificado, monto)
    VALUES (
      {int(val(row['anio']))}::integer,
      '{val(row['cod_siaf_area'])}'::varchar,
      '{str(val(row['siaf_fuente']))}'::varchar,
      '{val(row['certificado'])}'::varchar,
      {float(val(row['monto_clasificador']))}::numeric
    )
    ON CONFLICT DO NOTHING;
    """
    # Lo comprimimos en una sola línea si lo quieres así
    return query.replace("\n", " ")

# --- Uso ---

# Generas todos los INSERTs
insert_statements = [
    create_insert_consolidado(row)
    for _, row in df_1.iterrows()
]

# Opcional: los imprimes o los ejecutas
for sql in insert_statements:
    print(sql)
    # cur.execute(sql)  # si quieres ejecutarlos directamente
# conn.commit()


     INSERT INTO sistema_informacion_gerencial.hechos_institucional_consolidados       (anio, cod_siaf_area, fuente_siaf, num_certificado, monto)     VALUES (       2025::integer,       '0000'::varchar,       '00'::varchar,       '0000000001'::varchar,       8406955.48::numeric     )     ON CONFLICT DO NOTHING;     
     INSERT INTO sistema_informacion_gerencial.hechos_institucional_consolidados       (anio, cod_siaf_area, fuente_siaf, num_certificado, monto)     VALUES (       2025::integer,       '0000'::varchar,       '00'::varchar,       '0000000002'::varchar,       263809773.38::numeric     )     ON CONFLICT DO NOTHING;     
     INSERT INTO sistema_informacion_gerencial.hechos_institucional_consolidados       (anio, cod_siaf_area, fuente_siaf, num_certificado, monto)     VALUES (       2025::integer,       '0000'::varchar,       '00'::varchar,       '0000000003'::varchar,       11412972.0::numeric     )     ON CONFLICT DO NOTHING;     
     INSERT INTO sistema_informacion_gerenci

In [68]:
output_path = 'inserts.sql'

with open(output_path, 'w', encoding='utf-8') as f:
    # Escribimos cada sentencia en una línea nueva
    for sql in insert_statements:
        f.write(sql + '\n')

print(f"Archivo SQL generado en: {output_path}")

Archivo SQL generado en: inserts.sql
